In [1]:
"""
Test demo for MLP mapper.
"""
import argparse
import datetime
import h5py
import json
import os
import time
from pathlib import Path

import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
from transformers import AutoTokenizer, CLIPTextModel, BertTokenizer, BertModel

import util.misc as misc
from engine_node2node import get_text_embeddings
from util.datasets import build_shape_surface_occupancy_dataset

In [2]:
def get_args_parser():
    parser = argparse.ArgumentParser("Latent Diffusion", add_help=False)

    # Model parameters
    parser.add_argument(
        "--model",
        default="kl_d512_m512_l8_edm",
        type=str,
        metavar="MODEL",
        help="Name of model to train",
    )
    parser.add_argument(
        "--batch_size",
        default=32,
        type=int,
        help="Batch size per GPU (effective batch size is batch_size * accum_iter * # gpus",
    )
    parser.add_argument(
        "--text_model_name",
        type=str,
        help="Text model name to use",
    )
    parser.add_argument(
        "--ae",
        type=str,
        metavar="MODEL",
        help="Name of autoencoder",
    )
    parser.add_argument(
        "--ae-latent-dim",
        type=int,
        default=512*8,
        help="AE latent dimension",
    )
    parser.add_argument(
        "--ae_pth",
        required=True,
        help="Autoencoder checkpoint"
    )
    parser.add_argument(
        "--point_cloud_size",
        default=2048,
        type=int,
        help="input size"
    )
    parser.add_argument(
        "--fetch_keys",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--use_embeds",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--intensity_loss",
        action="store_true",
        default=False,
        help="Contrastive edit intensity loss using ground-truth labels.",
    )
    parser.add_argument(
        "--resume",
        default="",
        help="Resume from checkpoint"
    )
    parser.add_argument(
        "--resume_weights",
        action="store_true",
        default=False,
        help="Only resume weights, not optimizer state",
    )
    parser.add_argument(
        "--resume_full_weights",
        action="store_true",
        default=False,
        help="Resume the full model weights with the EDM wrapper",
    )

    # Dataset parameters
    parser.add_argument(
        "--dataset",
        type=str,
        choices=["graphedits"],
        help="dataset name",
    )
    parser.add_argument(
        "--ft_bert",
        action="store_true",
        default=False,
        help="Also fine-tune the BERT model",
    )
    parser.add_argument(
        "--data_path",
        type=str,
        help="dataset path",
    )
    parser.add_argument(
        "--data_type",
        type=str,
        help="dataset type",
    )
    parser.add_argument(
        "--max_edge_level",
        default=None,
        type=int,
        help="maximum edge level to use",
    )
    parser.add_argument(
        "--alt_ae_embeds",
        type=str,
        default=None,
        help="Alternative autoencoder embeddings to use",
    )
    parser.add_argument(
        "--device", default="cuda", help="device to use for training / testing"
    )
    parser.add_argument("--seed", default=0, type=int)
    parser.add_argument("--num_workers", default=60, type=int)
    parser.add_argument(
        "--pin_mem",
        action="store_true",
        help="Pin CPU memory in DataLoader for more efficient (sometimes) transfer to GPU.",
    )

    return parser

In [3]:
import models.mlp_mapper as mlp_mapper

# Set dummy arg string to debug the parser
call_string = """--ae_pth /ibex/user/slimhy/Shape2VecSet/output/pc_ae/best_model.pt \
    --ae-latent-dim 256 \
    --text_model_name bert-base-uncased \
    --dataset graphedits \
    --data_path /ibex/user/slimhy/ShapeWalk/ \
    --data_type release \
    --batch_size 32 \
    --num_workers 8 \
    --model mlp_mapper_bert_bneck_1024_pcae \
    --resume /ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_bneck_1024_pcae/checkpoint-90.pth \
    --resume_full_weights \
    --device cuda \
    --fetch_keys \
    --use_embeds \
    --alt_ae_embeds pc_ae \
    --seed 0"""

# Parse the arguments
args = get_args_parser()
args = args.parse_args(call_string.split())
args.use_clip = "clip" in args.text_model_name
device = torch.device(args.device)

model = mlp_mapper.__dict__[args.model](use_linear_proj=not args.use_clip)
model.to(device)

# Load the checkpoint
if args.resume:
    print("Loading checkpoint [%s]..." % args.resume)
    checkpoint = torch.load(args.resume, map_location="cpu")
    model.load_state_dict(checkpoint["model"])
    print("Done.")

Loading checkpoint [/ibex/user/slimhy/Shape2VecSet/output/graph_edit/dm/mlp_mapper_bert_bneck_1024_pcae/checkpoint-90.pth]...
Done.


In [ ]:
import models.autoencoders as autoencoders


# Instantiate autoencoder
ae = autoencoders.__dict__["kl_d512_m512_l8"]()
ae.eval()
ae_pth = "/ibex/user/slimhy/Shape2VecSet/output/graph_edit/ae/ae_m512.pth"
print("Loading autoencoder %s" % ae_pth)
ae.load_state_dict(torch.load(ae_pth, map_location="cpu")["model"])
ae.to(device)


In [ ]:
# --------------------

# Fix the seed for reproducibility
seed = args.seed + misc.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)

cudnn.benchmark = True

args.fetch_keys = True
dataset_train = build_shape_surface_occupancy_dataset("train", args=args)
dataset_val = build_shape_surface_occupancy_dataset("val", args=args)

# Create data loaders
data_loader_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=args.num_workers,
    pin_memory=True,
    drop_last=True,
)

data_loader_val = torch.utils.data.DataLoader(
    dataset_val,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=args.num_workers,
    pin_memory=True,
    drop_last=True,
)
# --------------------


In [ ]:
import json
import torch
import numpy as np
import os.path as osp
import pprint
import warnings

from argparse import ArgumentParser
from models.mlp import MLP
from models.point_net import PointNet
from models.pointcloud_autoencoder import PointcloudAutoencoder


def describe_pc_ae(args):
    # Make an AE.
    if args.encoder_net == "pointnet":
        ae_encoder = PointNet(init_feat_dim=3, conv_dims=args.encoder_conv_layers)
        encoder_latent_dim = args.encoder_conv_layers[-1]
    else:
        raise NotImplementedError()

    if args.decoder_net == "mlp":
        ae_decoder = MLP(
            in_feat_dims=encoder_latent_dim,
            out_channels=args.decoder_fc_neurons + [args.n_pc_points * 3],
            b_norm=False,
        )

    model = PointcloudAutoencoder(ae_encoder, ae_decoder)
    return model


def load_state_dicts(checkpoint_file, map_location=None, **kwargs):
    """ Load torch items from saved state_dictionaries"""
    if map_location is None:
        checkpoint = torch.load(checkpoint_file)
    else:
        checkpoint = torch.load(checkpoint_file, map_location=map_location)

    for key, value in kwargs.items():
        value.load_state_dict(checkpoint[key])

    epoch = checkpoint.get('epoch')
    if epoch:
        return epoch


def read_saved_args(config_file, override_or_add_args=None, verbose=False):
    """
    :param config_file: json file containing arguments
    :param override_args: dict e.g., {'gpu': '0'} will set the resulting arg.gpu to be 0
    :param verbose:
    :return:
    """
    parser = ArgumentParser()
    args = parser.parse_args([])
    with open(config_file, "r") as f_in:
        args.__dict__ = json.load(f_in)

    if override_or_add_args is not None:
        for key, val in override_or_add_args.items():
            args.__setattr__(key, val)

    if verbose:
        args_string = pprint.pformat(vars(args))
        print(args_string)

    return args


def load_pretrained_pc_ae(model_file):
    config_file = osp.join(osp.dirname(model_file), "config.json.txt")
    pc_ae_args = read_saved_args(config_file)
    pc_ae = describe_pc_ae(pc_ae_args)

    if osp.join(pc_ae_args.log_dir, "best_model.pt") != osp.abspath(model_file):
        warnings.warn(
            "The saved best_model.pt in the corresponding log_dir is not equal to the one requested."
        )

    best_epoch = load_state_dicts(model_file, model=pc_ae)
    print(f"Pretrained PC-AE is loaded at epoch {best_epoch}.")
    return pc_ae, pc_ae_args


# Instantiate autoencoder
print("Loading autoencoder [%s]..." % args.ae_pth)
pc_ae, pc_ae_args = load_pretrained_pc_ae(args.ae_pth)
pc_ae = pc_ae.to(device)
pc_ae = pc_ae.eval()
print("Done.")

In [ ]:
def apply_edit(net, x_a, x_b, embed_ab):
    # Reshape from (B, D, K) to (B, M)
    x_a = x_a.flatten(1)
    x_b = x_b.flatten(1)
    embed_ab = embed_ab.flatten(1)

    # Concatenate the latent vector with the embedding
    edit_vec = net(x_a, embed_ab)
    print(torch.mean(edit_vec)/torch.mean(x_a))
    # Add the edit vector to the latent vector
    return x_a + edit_vec

In [ ]:
import mcubes
import trimesh


@torch.inference_mode()
def encode_latent(pointcloud):   
    return ae.encode(pointcloud)
    

scale_factor = np.array([0.33408034, 0.39906635, 0.35794342])  # NORMALSHIET
rotate_matrix = np.array(
    [[1, 0, 0], [0, 1, 0], [0, 0, -1]]
)

# Apply both scaling and rotation
transform = lambda x: center_in_unit_sphere(np.matmul(
    rotate_matrix, x.T
).T) * scale_factor

# Inverse transform
inv_transform = lambda x: np.matmul(
    np.linalg.inv(rotate_matrix), x.T
).T / scale_factor


In [ ]:
for data in data_loader_val:
    break
edit_key, embeds_a, embeds_b, embeds_text = data

In [ ]:
# Move all the garbage to CUDA
embeds_a = embeds_a.cuda()
embeds_b = embeds_b.cuda()
embeds_text = embeds_text.cuda()
x_b_edited = apply_edit(model, embeds_a, embeds_b, embeds_text)
x_b = embeds_b
x_a = embeds_a

In [ ]:
dset_json = "/ibex/user/slimhy/ShapeWalk/release/release_val.json"
dset_json = json.load(open(dset_json))

In [ ]:
from plot_pc import plot_pointclouds

# Decode the batch
b_size = x_b.shape[0]
n_points = args.point_cloud_size

with torch.inference_mode():
    orig = pc_ae.decoder(x_a).reshape([b_size, 4096, 3])
    rec = pc_ae.decoder(x_b_edited).reshape([b_size, 4096, 3])
    rec_gt = pc_ae.decoder(x_b).reshape([b_size, 4096, 3])

In [ ]:
plot_pointclouds(orig[8:12].detach().cpu().numpy())

In [ ]:
# Visualize 8 reconstructions
plot_pointclouds(rec[8:12].detach().cpu().numpy())

In [ ]:
plot_pointclouds(rec_gt[8:12].detach().cpu().numpy())

In [ ]:
def transform_pc(in_pc):
    in_pc_detached = in_pc.detach().cpu().numpy()
    to_show = np.zeros_like(in_pc_detached)
    for k,pc in enumerate(in_pc_detached):
        to_show[k] = inv_transform(pc)
    return to_show

rec_t = transform_pc(rec)
plot_pointclouds(rec_t[8:12])

In [ ]:
orig_t = transform_pc(orig)
plot_pointclouds(orig_t[8:12])

In [22]:
rec_t = torch.tensor(rec_t).to(device)
orig_t = torch.tensor(orig_t).to(device)

_, latents_rec  = encode_latent(rec_t[:, :2048, :].contiguous())
_, latents_orig = encode_latent(orig_t[:, :2048, :].contiguous())

/tmp/ipykernel_27647/467312786.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rec_t = torch.tensor(rec_t).to(device)
/tmp/ipykernel_27647/467312786.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  orig_t = torch.tensor(orig_t).to(device)


In [23]:
@torch.inference_mode()
def decode_latent(shape_k, latent, fcode="", density=32):
    latent = latent.clone().cuda().reshape(1,512,8).type(torch.float32)
    density = density
    gap = 2. / density
    x = np.linspace(-1, 1, density+1)
    y = np.linspace(-1, 1, density+1)
    z = np.linspace(-1, 1, density+1)
    xv, yv, zv = np.meshgrid(x, y, z)
    grid = torch.from_numpy(np.stack([xv, yv, zv]).astype(np.float32)).view(3, -1).transpose(0, 1)[None].to(device, non_blocking=True)
 
    logits = ae.decode(latent, grid)
    logits = logits.detach()
    
    volume = logits.view(density+1, density+1, density+1).permute(1, 0, 2).cpu().numpy()
    verts, faces = mcubes.marching_cubes(volume, 0)
 
    verts *= gap
    verts -= 1
 
    m = trimesh.Trimesh(verts, faces)
    fname = 'decoded_shapes/node_%d_%s.obj' % (shape_k, fcode)
    m.export(fname)
    print("Written to [%s]." % fname)

In [24]:
# Decode a few of the edited latent vectors
for shape_k, latent_k in enumerate(latents_rec[8:12]):
    decode_latent(shape_k,
                  latent_k,
                  fcode="rec",
                  density=128)

Written to [decoded_shapes/node_0_rec.obj].
Written to [decoded_shapes/node_1_rec.obj].
Written to [decoded_shapes/node_2_rec.obj].
Written to [decoded_shapes/node_3_rec.obj].


In [25]:
for shape_k, latent_k in enumerate(latents_orig[8:12]):
    decode_latent(shape_k,
                  latent_k,
                  fcode="orig",
                  density=128)

Written to [decoded_shapes/node_0_orig.obj].
Written to [decoded_shapes/node_1_orig.obj].
Written to [decoded_shapes/node_2_orig.obj].
Written to [decoded_shapes/node_3_orig.obj].


In [26]:
def show_shape(shape_k):
    # First load all the meshes
    mesh_a = trimesh.load("decoded_shapes/node_%d_orig.obj" % shape_k)
    mesh_b = trimesh.load("decoded_shapes/node_%d_rec.obj" % shape_k)

    # Translate a to the left, b to the right
    mesh_a.apply_translation([-1, 0, 0])
    mesh_b.apply_translation([1, 0, 0])

    # Combine them into a single scene
    scene = trimesh.Scene([mesh_a, mesh_b])

    return scene.show()